In [5]:
# Step 1: Install Gurobi (Assuming Gurobi isn't pre-installed in your environment)
!pip install gurobipy

# Adjust the file path below
file_path = '/Users/saiarvindatluri/Downloads/Route755.xlsx'  # Update this path

# Step 3: Prepare the data
import pandas as pd
import numpy as np
from gurobipy import Model, GRB, quicksum

# Load the Excel file using the correct sheet names
store_details = pd.read_excel(file_path, sheet_name='StoreDetails')
time_matrix_df = pd.read_excel(file_path, sheet_name='Time')

# Convert the DataFrame into a dictionary where keys are (origin, destination) pairs and values are travel times
time_matrix = {
    (int(time_matrix_df.iloc[i]['ID No']), int(time_matrix_df.columns[j])): time_matrix_df.iloc[i, j]
    for i in range(len(time_matrix_df))
    for j in range(1, len(time_matrix_df.columns))
}

# Process 'store_details' to identify delivery days for each store
# Initialize a dictionary to keep track of delivery days for Costco and other stores
delivery_days = {}

# Define working days, excluding Wednesday and Sunday
working_days = ['Monday', 'Tuesday', 'Thursday', 'Friday', 'Saturday']

# Create a mapping for each store and whether it requires delivery on each working day
# This considers Costco's requirement for daily delivery on working days and the minimum 3 times a week for others
store_working_days = {}
for store_id, days in delivery_days.items():
    for day in working_days:
        # For Costco (identified by name or another criteria in your data), ensure delivery on all working days
        # For other stores, ensure a minimum of 3 deliveries a week based on your logic to identify these stores
        if 'Costco' in store_details.loc[store_details['ID No'] == store_id, 'Store'].values[0]:
            store_working_days[(store_id, day)] = True  # Costco stores require delivery on all working days
        else:
            # For other stores, this logic might need to be adjusted based on your specific requirements
            store_working_days[(store_id, day)] = day in days

# Prepare a matrix for loading times (assuming 15 minutes for each store)
loading_time = 15  # Loading time in minutes
working_hours_per_day = 9 * 60

# Extract store IDs, including the depot (assumed to be ID 0)
stores = sorted(store_details['ID No'].unique().tolist())

# Identify Costco stores from the 'Store' column
costco_stores = store_details[store_details['Store'].str.contains('Costco')]['ID No'].unique().tolist()

In [6]:
# Initialize the model
model = Model("VehicleRouteOptimization")

# Decision Variables
# Paths taken between stores on given days
visit_var = model.addVars(
    working_days, [(i, j) for i, j in time_matrix.keys() if i != j],
    vtype=GRB.BINARY, name="visit"
)

# Deliveries made to stores on given days
delivery_var = model.addVars(
    working_days, store_details['ID No'].unique(),
    vtype=GRB.BINARY, name="delivery"
)

### The Objective: Minimize Travel Time
model.setObjective(
    quicksum(visit_var[day, i, j] * time_matrix[(i, j)]
             for day in working_days for i, j in time_matrix.keys() if i != j),
    GRB.MINIMIZE
)


# Assuming depot is store 0
for day in working_days:
    # Ensure the depot is left first
    model.addConstr(quicksum(visit_var[day, 0, j] for j in stores if j != 0) == 1, f"StartFromDepot_{day}")
    # Ensure the depot is entered last
    model.addConstr(quicksum(visit_var[day, i, 0] for i in stores if i != 0) == 1, f"ReturnToDepotLast_{day}")

# Single exit and entry for each store, ensuring path continuity
for day in working_days:
    for store in stores[1:]:  # Excluding the depot
        model.addConstr(quicksum(visit_var[day, i, store] for i in stores if i != store) >= delivery_var[day, store], f"MinIncoming_{day}_{store}")
        model.addConstr(quicksum(visit_var[day, store, j] for j in stores if j != store) >= delivery_var[day, store], f"MinOutgoing_{day}_{store}")

# Delivery requirements for Costco and other stores
for store in stores[1:]:
    if store in costco_stores:
        for day in working_days:
            model.addConstr(delivery_var[day, store] == 1, f"CostcoDelivery_{day}_{store}")
    else:
        model.addConstr(quicksum(delivery_var[day, store] for day in working_days) >= 3, f"MinDeliveries_{store}")

# Prevent delivery after two consecutive days with a more flexible approach
for store in stores:
    for i in range(len(working_days) - 2):
        day1, day2, day3 = working_days[i], working_days[i + 1], working_days[i + 2]
        model.addConstr((delivery_var[day1, store] + delivery_var[day2, store] + (1 - delivery_var[day3, store])) >= 1,
                        f"FlexibleNoConsecutiveDelivery_{store}_{day1}_{day2}_{day3}")

# Constraint to limit the number of deliveries per day to less than 19
for day in working_days:
    model.addConstr(
        quicksum(delivery_var[day, store] for store in stores) <= 18,
        f"MaxDeliveriesPerDay_{day}"
    )
    
# Working hours constraint, adjusted for minimized total travel time
for day in working_days:
    model.addConstr(
        quicksum((time_matrix[(i, j)] + loading_time) * visit_var[day, i, j] for i in stores for j in stores if i != j) <= working_hours_per_day,
        f"WorkingHours_{day}"
    )


Set parameter Username
Academic license - for non-commercial use only - expires 2024-12-12


In [7]:
# Solve the optimization model
model.optimize()

# Placeholder for daily store visits based on Gurobi model output
daily_visits = {day: [] for day in working_days}

# Extracting visits from the model's solution
for day in working_days:
    for i, j in [(i, j) for i in stores for j in stores if i != j]:
        if visit_var[day, i, j].X > 0.5:  # Threshold to consider if the route is selected
            if i not in daily_visits[day]:
                daily_visits[day].append(i)
            if j not in daily_visits[day]:
                daily_visits[day].append(j)

# Function to sequence the store visits
def format_sequence(sequence):
    """
    Format the sequence of visits to display transitions between stores and back to the depot.
    """
    # Ensure the sequence ends with the depot for a complete loop
    if sequence[-1] != 0:
        sequence.append(0)

    # Build the formatted string
    transitions = [f"store {sequence[i]} to store {sequence[i+1]}" for i in range(len(sequence) - 1)]
    formatted_sequence = "; ".join(transitions)
    return formatted_sequence

# Apply formatting to the sequenced visits
for day, stores in daily_visits.items():
    formatted_sequence = format_sequence(stores)
    print(f"Day: {day}, Route: {formatted_sequence}")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 423 rows, 4205 columns and 13006 nonzeros
Model fingerprint: 0xdea151b5
Variable types: 0 continuous, 4205 integer (4205 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [1e+00, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 19 rows and 15 columns
Presolve time: 0.02s
Presolved: 404 rows, 4190 columns, 12924 nonzeros
Variable types: 0 continuous, 4190 integer (4190 binary)

Root relaxation: objective 6.932000e+02, 916 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  693.20000    0    8          -  693.20000      -     -    0s
H    0     0           